# Mock Data Generation and Data Preparation for ingestion in Neo4j
---

The mock data is generated using simple and humble tools like Excel, and some creative help from the internet ([Mockaroo](https://mockaroo.com/)).

The data resembles material consumption for a Job/Work Order, projected on a exploded BOM-like structure. The data is exported to CSV format and stored in the `data` directory.

This data is then converted into separate CSV files to ingest into Neo4j database using humble Python and pandas magic. The data is stored in the `data/neo4j` directory. The list of CSV files are:
1. items.csv `(:item)`
2. lots.csv `(:lot)`
3. jobs.csv `(:job)`
4. item_belongs_to_lot.csv `(:item)-[:BELONGS_TO]->(:lot)`
5. lot_consumed_by_job.csv `(:lot)-[:CONSUMED_BY]->(:job)`
6. job_produces_lot.csv `(:job)-[:PRODUCES]->(:lot)`

In [5]:
# importing required libraries
import os
from pathlib import Path

import pandas as pd
import numpy as np

# setting the home directory
path_home = Path('..')
path_export = path_home / 'data' / 'neo4j'
os.mkdir(path_export) if not os.path.exists(path_export) else None

In [6]:
df_raw = pd.read_csv(path_home / 'data' / 'data.csv')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   L1_ItemNumber       674 non-null    object
 1   L1_ItemDescription  674 non-null    object
 2   L1_ItemType         674 non-null    object
 3   L1_LotNumber        674 non-null    object
 4   L1_Job              674 non-null    object
 5   L1_JobStatus        674 non-null    object
 6   L2_Quantity         674 non-null    int64 
 7   L2_ItemNumber       674 non-null    object
 8   L2_ItemDescription  674 non-null    object
 9   L2_ItemType         674 non-null    object
 10  L2_LotNumber        674 non-null    object
 11  L2_Job              674 non-null    object
 12  L2_JobStatus        674 non-null    object
 13  L3_Quantity         674 non-null    int64 
 14  L3_ItemNumber       674 non-null    object
 15  L3_ItemDescription  674 non-null    object
 16  L3_ItemType         674 no

In [7]:
def getNodeDF(df, cols, levels):
    """
    This function takes a dataframe and returns a new dataframe with unique values for the speciied cols and levels.
    """
    newcols_item = cols
    list_df_items = []
    
    for LEVEL in levels:
        oldcols_item = [f'L{str(LEVEL)}_{col}' for col in newcols_item]
        
        list_df_items.append(
            df_raw \
                .loc[:, oldcols_item] \
                .rename(columns=dict(zip(oldcols_item, newcols_item)))
        )
    
    return pd.concat(list_df_items, ignore_index=True) \
        .drop_duplicates() \
        .reset_index(drop=True)

## Extracting Node Data
---


### (:Item)
| item_number | item_description | item_type |
|-------------|------------------|-----------|

---

- To extract item details, the ItemNumber, ItemDescription, and ItemType fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (ItemNumber, ItemDescription, ItemType) are extracted
- The unique values are stored in the `items.csv` file

In [8]:
df_item = getNodeDF(
    df      = df_raw,
    cols    = ['ItemNumber', 'ItemDescription', 'ItemType',],
    levels  = [1, 2, 3, 4, 5]
)
df_item.to_csv(path_export / 'item.csv', index=False)
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ItemNumber       875 non-null    object
 1   ItemDescription  875 non-null    object
 2   ItemType         875 non-null    object
dtypes: object(3)
memory usage: 20.6+ KB


### (:Lot)
| ItemNumber | LotNumber |
|------------|-----------|

---

- To extract lot details, the ItemNumber, LotNumber fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (ItemNumber, LotNumber) are extracted
- The unique values are stored in the `lots.csv` file

In [9]:
df_lot = getNodeDF(
    df      = df_raw,
    cols    = ['ItemNumber', 'LotNumber',],
    levels  = [1, 2, 3, 4, 5]
)
df_lot.to_csv(path_export / 'lot.csv', index=False)
df_lot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370 entries, 0 to 3369
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ItemNumber  3370 non-null   object
 1   LotNumber   3370 non-null   object
dtypes: object(2)
memory usage: 52.8+ KB


### (:Job)
| Job | JobStatus |
|-----|-----------|

---

- To extract job details, the Job, JobStatus fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (Job, JobStatus) are extracted
- The unique values are stored in the `jobs.csv` file

In [10]:
df_job = getNodeDF(
    df      = df_raw,
    cols    = ['Job', 'JobStatus',],
    levels  = [1, 2, 3, 4]
)
df_job.to_csv(path_export / 'job.csv', index=False)
df_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Job        2121 non-null   object
 1   JobStatus  2121 non-null   object
dtypes: object(2)
memory usage: 33.3+ KB


## Extracting Edge Data
---

In [11]:
def getEdgeDF(df, cols, levels, adds):
    """
    This function takes a dataframe and returns a new dataframe with unique values for the speciied cols and levels.
    """
    newcols_item = cols
    list_df_items = []
    
    for LEVEL in levels:
        oldcols_item = [f'L{str(LEVEL+add)}_{col}' for col,add in zip(newcols_item, adds)]
        
        list_df_items.append(
            df_raw \
                .loc[:, oldcols_item] \
                .rename(columns=dict(zip(oldcols_item, newcols_item)))
        )
    
    return pd.concat(list_df_items, ignore_index=True) \
        .drop_duplicates() \
        .reset_index(drop=True)

### (:Item)-[:BELONGS_TO]->(:Lot)
| ItemNumber | LotNumber |
|------------|-----------|

---

- To extract the relationship between Item and Lot, the ItemNumber, LotNumber fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (ItemNumber, LotNumber) are extracted
- The unique values are stored in the `item_belongs_to_lot.csv` file

In [12]:
df_item_belongs_to_lot = getEdgeDF(
    df      = df_raw,
    cols    = ['ItemNumber', 'LotNumber',],
    levels  = [1, 2, 3, 4, 5],
    adds    = [0, 0]
)
df_item_belongs_to_lot.to_csv(path_export / 'item_belongs_to_lot.csv', index=False)
df_item_belongs_to_lot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370 entries, 0 to 3369
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ItemNumber  3370 non-null   object
 1   LotNumber   3370 non-null   object
dtypes: object(2)
memory usage: 52.8+ KB


### (:Lot)-[:CONSUMED_BY]->(:Job)
| ItemNumber | LotNumber | Job | Quantity |
|------------|-----------|-----|----------|

---

- To extract the relationship between Lot and Job, the (L)ItemNumber, (L)LotNumber, (L+1)Job, (L)Quantity fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (ItemNumber, LotNumber, Job, Quantity) are extracted
- The unique values are stored in the `lot_consumed_by_job.csv` file

In [13]:
df_lot_consumed_by_job = getEdgeDF(
    df      = df_raw,
    cols    = ['ItemNumber', 'LotNumber', 'Job', 'Quantity'],
    levels  = [1, 2, 3, 4],
    adds    = [1, 1, 0, 1]
)
df_lot_consumed_by_job.to_csv(path_export / 'lot_consumed_by_job.csv', index=False)
df_lot_consumed_by_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ItemNumber  2696 non-null   object
 1   LotNumber   2696 non-null   object
 2   Job         2696 non-null   object
 3   Quantity    2696 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 84.4+ KB


### (:Job)-[:PRODUCES]->(:Lot)
| Job | ItemNumber | LotNumber |
|-----|------------|-----------|

---

- To extract the relationship between Job and Lot, the (L)Job, (L)ItemNumber, (L)LotNumber fields are concatenated (L1, L2, L3, L4, L5 levels)
- unique values of (Job, ItemNumber, LotNumber) are extracted
- The unique values are stored in the `job_produces_lot.csv` file

In [14]:
df_job_produces_lot = getEdgeDF(
    df      = df_raw,
    cols    = ['Job', 'ItemNumber', 'LotNumber'],
    levels  = [1, 2, 3, 4],
    adds    = [0, 0, 0]
)
df_job_produces_lot.to_csv(path_export / 'job_produces_lot.csv', index=False)
df_job_produces_lot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Job         2696 non-null   object
 1   ItemNumber  2696 non-null   object
 2   LotNumber   2696 non-null   object
dtypes: object(3)
memory usage: 63.3+ KB
